In [2]:
import os 
import sys
import backtrader as bt
import pandas as pd
import pyfolio as pf

# 取得當前檔案的所在目錄
current_dir = os.path.dirname(os.path.abspath(__name__))
parent_dir = os.path.dirname(os.path.join(current_dir))

/opt/homebrew/Caskroom/miniforge/base/envs/stock/lib/python3.13/site-packages/pyfolio/pos.py:25: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
current_notebook_dir = os.getcwd()
project_root_path = os.path.dirname(os.path.dirname(current_notebook_dir))
sys.path.append(project_root_path)

import Chapter1.utils as chap1_utils

In [4]:
chap1_utils.finlab_login()

輸入成功!


In [5]:
analysis_period_start_date = "2017-05-16"
analysis_period_end_date = "2021-05-16"

In [6]:
top_N_stocks = chap1_utils.get_top_stocks_by_market_value(
    excluded_industry=[
        "金融業",
        "金融保險業",
        "存託憑證",
        "建材營造",
    ],
    pre_list_date="2017-01-03",
)

Due to your status as a free user, the most recent data has been shortened or limited.
Daily usage: 0.9 / 500 MB - company_basic_info


In [9]:
# 取得指定股票代碼列表在給定日期範圍內的每日 OHLCV 數據
all_stock_data = chap1_utils.get_daily_OHLCV_data(
    stock_symbols=top_N_stocks,
    start_date=analysis_period_start_date,
    end_date=analysis_period_end_date
)
# all_stock_data["datetime"] = all_stock_data["datetime"].astype(str)

/Users/houguanyu/Documents/code/python/stock/Quant/PythonQuantrading/Chapter1/utils.py:476: FutureWarning: YF.download() has changed argument auto_adjust default to True
  pd.DataFrame(yf.download(symbol, start=start_date, end=end_date)).droplevel(
[*********************100%***********************]  1 of 1 completed
/Users/houguanyu/Documents/code/python/stock/Quant/PythonQuantrading/Chapter1/utils.py:476: FutureWarning: YF.download() has changed argument auto_adjust default to True
  pd.DataFrame(yf.download(symbol, start=start_date, end=end_date)).droplevel(
[*********************100%***********************]  1 of 1 completed
/Users/houguanyu/Documents/code/python/stock/Quant/PythonQuantrading/Chapter1/utils.py:476: FutureWarning: YF.download() has changed argument auto_adjust default to True
  pd.DataFrame(yf.download(symbol, start=start_date, end=end_date)).droplevel(
[*********************100%***********************]  1 of 1 completed
/Users/houguanyu/Documents/code/python/stock/Q

In [14]:
all_stock_data["datetime"] = all_stock_data["datetime"].astype(str)
all_stock_data["asset"] = all_stock_data["asset"].astype(str)

In [16]:
# 指定各個季度下要使用來排序的因子
# name 對應的是每個季度的因子排名
# corr 對應的是因子值與未來收益的關係（根據單因子Alaphalens分析結果）。
select_rank_factor_dict = {
    "2017-Q1": {"name": "稅後淨利成長率", "corr": True},
    "2017-Q2": {"name": "稅後淨利成長率", "corr": True},
    "2017-Q3": {"name": "稅後淨利成長率", "corr": True},
    "2017-Q4": {"name": "稅後淨利成長率", "corr": True},
    "2018-Q1": {"name": "稅前淨利成長率", "corr": True},
    "2018-Q2": {"name": "稅前淨利成長率", "corr": True},
    "2018-Q3": {"name": "稅前淨利成長率", "corr": True},
    "2018-Q4": {"name": "稅前淨利成長率", "corr": True},
    "2019-Q1": {"name": "稅後淨利成長率", "corr": True},
    "2019-Q2": {"name": "稅後淨利成長率", "corr": True},
    "2019-Q3": {"name": "稅後淨利成長率", "corr": True},
    "2019-Q4": {"name": "稅後淨利成長率", "corr": True},
    "2020-Q1": {"name": "稅前淨利成長率", "corr": True},
    "2020-Q2": {"name": "稅前淨利成長率", "corr": True},
    "2020-Q3": {"name": "稅前淨利成長率", "corr": True},
    "2020-Q4": {"name": "稅前淨利成長率", "corr": True},
}

In [ ]:
# 準備因子數據，將各個季度的因子數據進行排序
all_factor_data = pd.DataFrame()
for quarter, factor in select_rank_factor_dict.items():
    # 將季度字串轉換為起始和結束日期
    start_date, end_date = chap1_utils.convert_quarter_to_dates(quarter)    
    # 生成該季度的交易範圍
    trading_days = pd.date_range(start=start_date, end=end_date)
    # 取得因子數據，並按照股票代碼和日期進行排序與填補
    quarter_factor_data = (
        chap1_utils.get_factor_data(
            stock_symbols=top_N_stocks,
            factor_name=factor["name"],
            trading_days=list(trading_days),
        )
        .reset_index()
        .assign(factor_name=factor["name"])
        .sort_values(by=["asset", "datetime"])
        .groupby("asset", group_keys=False)
        .apply(lambda group: group.ffill())
        .dropna()
    )
    # 根據因子值進行股票排序: 由小到大(positive_corr=True) or 由大到小(possitive_corr=False)
    quarter_factor_data = chap1_utils.rank_stocks_by_factor(
        factor_df=quarter_factor_data,
        positive_corr=factor["corr"], # 根據因子相關性決定排序方向
        rank_column="value", # 用來排序的欄位名稱
        rank_result_column="rank", # 儲存排序結果的欄位名稱
    ).drop(columns=["value"])
    # 合併該季度的因子數據
    all_factor_data = pd.concat([all_factor_data, quarter_factor_data])
# 重設索引並將日期與股票代碼轉換為字串格式